In [70]:
import os, json, numpy as np, pandas as pd
import seaborn as sns, matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from tweepy.api import API
from tweepy.auth import OAuthHandler
from tweepy.cursor import Cursor
from tweepy.streaming import Stream, StreamListener
from pprint import PrettyPrinter, pformat, pprint
from TweepyCredentials import * 

In [71]:
sns.set_style('whitegrid')
%matplotlib inline
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [72]:
class Authenticator():
    '''A class that authenticates out API call.'''
    
    def authenticate_tweets(self):
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_KEY_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth

In [73]:
class MyListener(StreamListener):
    '''A custom listener class inheriting from tweepy StreamListener class methods that print/write streamed tweets.'''
    
    def __init__(self, tweet_file):
        self.tweet_file = tweet_file
        self.auth = Authenticator().authenticate_tweets()
    
    def on_data(self, status):
        try:
            # main goal is to append tweets to a list or write the tweets to a file
            tweet = json.loads(status)
            with open(self.tweet_file, 'a') as tf:
                tf.write(tweet['text'])
        except BaseException as e:
            print('Error: ', str(e))

    def on_error(self, error):
        print(error)
        if error == 420:
            return False
        

In [74]:
class TwitterClient():
    '''A class to get various data from a specified user'''
    
    def __init__(self):
        self.auth = Authenticator().authenticate_tweets()
        self.client = API(self.auth)
        
    def get_user_tweets(self, num_tweets, id_):
        tweets = []
        for tweet in Cursor(self.client.user_timeline, id_).items(num_tweets):
            tweets.append(tweet._json)
            
        delete = []
        for tweet in tweets:
            for key, value in tweet.items():
                if value in [None, False]:
                    delete.append(key)    

        for key in delete:
            if key in tweet:
                del tweet[key]
            else:
                pass    
            
        return tweets

In [ ]:
if __name__ == "__main__":
    
    from nltk.corpus import stopwords
    
    client = TwitterClient()
    
    tweets = client.get_user_tweets(100, 'realDonaldTrump')
    tweet_text = [tweet['text'] for tweet in tweets]
    retweet_count = [tweet['retweet_count'] for tweet in tweets]
    created_at = [tweet['created_at'] for tweet in tweets]
    
    df['Created At'] = created_at
    df['Retweets'] = retweet_count
    df['Created At'] = df['Created At']
    
#     fig = plt.figure(figsize=(15,4))
#     axes = fig.add_axes([0,0,1,1])
#     axes.plot(df['Created At'], df['Retweets'], label = 'Retweet trend with time')
    
    stopwords = stopwords.words('english')
    added_stopwords = ('a', 'for', '@', 'the', 'with', ',','.','be','to','I')
    
    for word in added_stopwords:
        stopwords.append(word)
  
    stopwords = set(stopwords)
#     print(stopwords)
    
    tweet_tokens = [nltk.word_tokenize(tweet) for tweet in tweet_text]
#     print(pprint(tweet_tokens[1]))
    
    filtered_stopwords = []
    
    for token in tweet_tokens:
        for word in token:
            if word in stopwords:
                filtered_stopwords.append(word)
                token.remove(word)
    print(len(filtered_stopwords))
    